In [ ]:
import copy
import sys
sys.path.insert(0, "..")
import numpy as np
import matplotlib.pyplot as plt
from classes.calculators.lennardjones import LennardJones
from classes.regression.regressers import pol_kernel_func, RBF_kernel_func, pol_kernel_regressor

In [ ]:
def dot_prod(x1, x2):
    x1 = np.array([1.0, np.sqrt(2)*x1, x1**2])
    x_dat = np.zeros(shape=(len(x2), 3))
    x_dat[:,0]+=1.0
    x_dat[:,1]+=np.sqrt(2)*x2
    x_dat[:,2]+=x2*x2
    return np.dot(x1, x_dat.T)


In [ ]:
xs = np.linspace(-5.0, 5.0, 1000)
fig, axs = plt.subplots(1,2, figsize=(13,5))
ys_dot = dot_prod(2.0, xs)
ys_kern = pol_kernel_func(x_basis=np.array([2.0]), x_data=xs, N_deg=2).reshape(1,-1)[0]
for ax in axs:
    ax.grid()
axs[0].plot(xs, ys_kern)
axs[0].plot(xs, ys_dot, "--")
xs1 = np.random.rand(20)*10 - 5.0
xs2 = np.random.rand(20)*10 - 5.0
ys_dot_tot = []
ys_kern_tot = []
for x1 in xs1:
    ys_dot = dot_prod(x1=x1, x2=xs2)
    ys_kern = pol_kernel_func(x_basis=np.array([x1]), x_data=xs2, N_deg=2).reshape(1,-1)[0]
    ys_dot_tot.append(ys_dot)
    ys_kern_tot.append(ys_kern)

axs[1].scatter(np.array(ys_dot_tot).flatten(), np.array(ys_kern_tot).flatten(), alpha=0.3, s=20)

In [ ]:
class DiscreteRBF():
    def __init__(self, delta_x, sigma=1.0, color="C4") -> None:
        self.delta_x = delta_x
        self.color = color
        self.sigma = sigma
        self.sig_phi = self.sigma/np.sqrt(2.0)
        self.A = np.sqrt(np.sqrt(2.0*np.pi)*self.sigma)
        self.bin_edges = np.arange(-12,12.01, self.delta_x)
        self.bin_centers = (self.bin_edges[:-1] + self.bin_edges[1:]) /2

    def RBF(self, xs, x_val):
        t1 = 1.0/(np.sqrt(2.0*np.pi)*self.sig_phi)
        t2 = np.exp(-(xs-x_val)**2/(2.0*self.sig_phi**2))
        return self.A*t1*t2*np.sqrt(self.delta_x)

    def get_descriptor(self, x_val, return_bins=False):
        return self.RBF(xs=self.bin_centers, x_val=x_val)
    
    def plot(self, x_val, ax):
        desc_vect =self.get_descriptor(x_val, return_bins=True)
        ax.bar(self.bin_centers,desc_vect,width=0.8*self.delta_x,color=self.color)
        ax.xaxis.set_major_locator(plt.MultipleLocator(1.0))


In [ ]:
x1 = 0.25
x2 = 0.5
for dx in [0.1,0.02,0.02, 0.04, 0.008, 0.0016, 0.00032, 0.000064]:
    discrete_rbf = DiscreteRBF(delta_x=dx, sigma=0.1)
    desc = discrete_rbf.get_descriptor(x1)
    #print(desc.round(3))
    print(discrete_rbf.get_descriptor(x1) @ discrete_rbf.get_descriptor(x1),
          discrete_rbf.get_descriptor(x2) @ discrete_rbf.get_descriptor(x2),
          discrete_rbf.get_descriptor(x1) @ discrete_rbf.get_descriptor(x2))

In [ ]:
fig, axes = plt.subplots(1,4,figsize=(16,4))
for ax, dx in zip(axes,[0.2,0.15,0.1,0.05]):
    discrete_rbf = DiscreteRBF(delta_x=dx, sigma=1e-1, color="C0")
    discrete_rbf.plot(0.5,ax)
    discrete_rbf.color="C1"
    discrete_rbf.plot(0.25,ax)
    ax.set_xlim([-1,1])
    ax.text(0.05,0.8,f'$\Delta x={dx:.2f}$',transform = ax.transAxes)